# Testing screencapture with different libraries
   (openCV, PyAutoGUI, Pillow, PyWin32)
    
### References:
Screen capture:
* https://stackoverflow.com/questions/24129253/screen-capture-with-opencv-and-python-2-7
* https://www.youtube.com/watch?v=WymCpVUPWQ4&ab_channel=LearnCodeByGaming
* https://www.youtube.com/watch?v=ks4MPfMq8aQ&ab_channel=sentdex
* https://stackoverflow.com/questions/3586046/fastest-way-to-take-a-screenshot-with-python-on-windows
* https://stackoverflow.com/questions/41785831/how-to-optimize-conversion-from-pycbitmap-to-opencv-image

Window listing:
* https://stackoverflow.com/questions/55547940/how-to-get-a-list-of-the-name-of-every-open-window

### Importing Libraries
Note: Original PIL (Python Imaging Library) was discontinued since 2011, with Pillow taking it's place.

In [5]:
import cv2 as cv
import numpy as np
from time import time


#import pyautogui # needs to be installed first
#from mms import mms # needs to be installed first

### Main Loop (With Pillow):

In [53]:
from PIL import ImageGrab   # This is actually Pillow, but it's still imported as PIL. 
counter = 1
total = 0

while(True):
    prev_time = time()
    
    #take the screenshot, bbox defines a boundry box for it
    bbox = (0, 0, 320, 240)
    #bbox = None
    img_pil = ImageGrab.grab(bbox)
    #transform to an array and reshape it into a matrix for openCV
    img_np = np.array(img_pil, dtype='uint8').reshape((img_pil.size[1], img_pil.size[0],3))
    
    img_np = cv.cvtColor(img_np, cv.COLOR_RGB2BGR)
    
    cv.imshow('window',img_np)
    #print('FPS {}'.format(1 / (time() - prev_time)))
    total += 1 / (time() - prev_time)
    print('Avg FPS {}'.format(total/counter))
    counter += 1
    if cv.waitKey(1) & 0xFF == ord('q'):
        cv.destroyAllWindows()
        break

Avg FPS 1.0839529731858673
Avg FPS 11.667628353731402
Avg FPS 14.062613515612137
Avg FPS 14.927178108295099
Avg FPS 15.338847786027818
Avg FPS 16.65890253558131
Avg FPS 17.132069213956495
Avg FPS 16.83432089249981
Avg FPS 17.106278571877137
Avg FPS 17.725651325756402
Avg FPS 18.18733554563593
Avg FPS 18.57065795438509
Avg FPS 18.638323585884198
Avg FPS 18.8308940397881
Avg FPS 19.05503604002528
Avg FPS 19.24194508072864
Avg FPS 19.358225844536523
Avg FPS 19.755648274739638
Avg FPS 19.95448919953419
Avg FPS 20.163446174581505
Avg FPS 20.273817700768934
Avg FPS 20.80596091240868
Avg FPS 21.07726985301259
Avg FPS 21.222508562530876
Avg FPS 21.240343217549892
Avg FPS 21.218573345942623
Avg FPS 21.413640204577177
Avg FPS 21.482775393417892
Avg FPS 21.571996820639303
Avg FPS 21.574318720292457
Avg FPS 21.5101500964485
Avg FPS 21.53028103795295
Avg FPS 21.60213565300189
Avg FPS 21.431026898157384
Avg FPS 21.642766038162247
Avg FPS 21.72242031493191
Avg FPS 21.70147747053759
Avg FPS 21.6766632

Avg FPS 22.87722287342148
Avg FPS 22.879379960108604
Avg FPS 22.87810857103847
Avg FPS 22.893687579218692
Avg FPS 22.900640474539276
Avg FPS 22.903265695120005
Avg FPS 22.900967668053962
Avg FPS 22.9253880742867
Avg FPS 22.93909123814982
Avg FPS 22.944353680436976
Avg FPS 22.944635174713717
Avg FPS 22.94051485922792
Avg FPS 22.953307602648074
Avg FPS 22.95875231817897
Avg FPS 22.958537358479685
Avg FPS 22.950105871348384
Avg FPS 22.963460761037695
Avg FPS 22.971159539257712
Avg FPS 22.972351915269343
Avg FPS 22.97000206270743
Avg FPS 22.987851284092677
Avg FPS 22.99814448714841
Avg FPS 23.001634889194943
Avg FPS 22.999606840753767
Avg FPS 22.993736394134782
Avg FPS 23.00788759340598
Avg FPS 23.015341754398094
Avg FPS 23.00775215897139
Avg FPS 23.001917519730792
Avg FPS 23.01873007279899
Avg FPS 23.02727251131637
Avg FPS 23.02778094113087
Avg FPS 23.024434348271498
Avg FPS 23.045519692599573
Avg FPS 23.05861084003209
Avg FPS 23.065001089714453
Avg FPS 23.06421527707206
Avg FPS 23.059396

Avg FPS 23.281111600524962
Avg FPS 23.27054526689454
Avg FPS 23.268996083715734
Avg FPS 23.27031206707035
Avg FPS 23.27103488867602
Avg FPS 23.260614644879116
Avg FPS 23.27228671195767
Avg FPS 23.279561162164512
Avg FPS 23.281690036180823
Avg FPS 23.28132551135694
Avg FPS 23.268727475600556
Avg FPS 23.272381308009674
Avg FPS 23.269459217638023
Avg FPS 23.26789480306142
Avg FPS 23.262548788395968
Avg FPS 23.26904491307815
Avg FPS 23.27194495404931
Avg FPS 23.26174166865348
Avg FPS 23.25792811762552
Avg FPS 23.254497011813005
Avg FPS 23.259141274809508
Avg FPS 23.259380268567337
Avg FPS 23.25662266837677
Avg FPS 23.25252458117795
Avg FPS 23.258338481074638
Avg FPS 23.260800010818862
Avg FPS 23.259112894209114
Avg FPS 23.25826935374885
Avg FPS 23.254570866706754
Avg FPS 23.24959460938367
Avg FPS 23.24558739193244
Avg FPS 23.253363786707457
Avg FPS 23.258607712833314
Avg FPS 23.25693620475193
Avg FPS 23.25447492075352
Avg FPS 23.24334762916812
Avg FPS 23.243460363410765
Avg FPS 23.24496721

Avg FPS 23.221192895735857
Avg FPS 23.223750449971963
Avg FPS 23.22522082711987
Avg FPS 23.225274973543854
Avg FPS 23.222804154955845
Avg FPS 23.228447816908236
Avg FPS 23.230688950322783
Avg FPS 23.23079829055869
Avg FPS 23.228985275030414
Avg FPS 23.225806349245925
Avg FPS 23.229666936629386
Avg FPS 23.230560187270818
Avg FPS 23.22919881718142
Avg FPS 23.23630062110537
Avg FPS 23.24064686027182
Avg FPS 23.24221147211047
Avg FPS 23.24147430379571
Avg FPS 23.23309713614245
Avg FPS 23.237176559817183
Avg FPS 23.239006504293453
Avg FPS 23.23338676122919
Avg FPS 23.230829883961476
Avg FPS 23.23364287099026
Avg FPS 23.233345085307107
Avg FPS 23.233030763986022
Avg FPS 23.230589023330502
Avg FPS 23.23355488625537
Avg FPS 23.2346176873343
Avg FPS 23.232680848253473
Avg FPS 23.238624678085017
Avg FPS 23.243108390412736
Avg FPS 23.244421859923083
Avg FPS 23.24381969021501
Avg FPS 23.241880197139277
Avg FPS 23.2472900616016
Avg FPS 23.24967698431027
Avg FPS 23.25001253458792
Avg FPS 23.24738103

#### Pillow results:
The framerate seems awfully slow, and reducing the boundry box to ridiculously small sizes barely seems to help.


### With Windows API:

In [55]:
import win32gui
import win32ui
import win32con

def window_capture():
    #w = 1920 # set this
    #h = 1080 # set this
    bmpfilenamename = "sample.bmp" #set this

    hwnd = win32gui.FindWindow(None, 'Samurai Gunn')
    #hwnd = None
    
    # Getting the window's size and accounting for window screenshot borders
    window_rect = win32gui.GetWindowRect(hwnd)
    titlebar_px = 38
    border_px = 9
    w = window_rect[2] - window_rect[0] - (2 * border_px)
    h = window_rect[3] - window_rect[1] - titlebar_px - border_px
    crop_x = border_px
    crop_y = titlebar_px
    
    wDC = win32gui.GetWindowDC(hwnd)
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (w, h) , dcObj, (crop_x, crop_y), win32con.SRCCOPY)
    
    # saving screenshot to file
    #dataBitMap.SaveBitmapFile(cDC, bmpfilenamename)
    
    # Converting to format useful for opencv
    signedIntsArray = dataBitMap.GetBitmapBits(True)
    #img = np.fromstring(signedIntsArray, dtype='uint8')
    # np.fromstring is deprecated, should use np.frombuffer instead
    img = np.frombuffer(signedIntsArray, dtype='uint8')
    img.shape = (h, w, 4)

    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC)
    win32gui.DeleteObject(dataBitMap.GetHandle())
    
    # Dropping alpha channel may be useful for some applications, like cv.matchTemplate()
    # which may throw an error otherwise
    #img = img[...,:3]   # this drops alpha channel 
    
    return img

#window_capture()

In [ ]:
import win32gui
# If we need help to figure out the window name, this funcion will list the names of every open window
def list_all_open_windows():
    def winEnumHandler( hwnd, ctx ):
        if win32gui.IsWindowVisible( hwnd ):
            print (hex(hwnd), win32gui.GetWindowText( hwnd ))

    win32gui.EnumWindows( winEnumHandler, None )

list_all_open_windows()

In [3]:
from pywinauto import Desktop

windows = Desktop(backend="uia").windows()
print([w.window_text() for w in windows])

['Taskbar', 'reading_from_screen - Jupyter Notebook - Google Chrome', 'Samurai Gunn', 'Anaconda Prompt (anaconda3)', 'Spotify Premium', 'Calculator', 'experiments', 'NVIDIA GeForce Overlay', 'Jupyter Notebook (anaconda3)', 'general - Discord', 'Steam', '', 'Program Manager']


## Main Loop (with pywin32)

In [54]:
counter = 1
total = 0

while(True):
    prev_time = time()
    screenshot = window_capture()
    
    cv.imshow('Screenshot', screenshot)
    #print('FPS {}'.format(1 / (time() - prev_time)))
     
    total += 1 / (time() - prev_time)
    print('Avg FPS {}'.format(total/counter))
    counter += 1
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        cv.destroyAllWindows()
        break

Avg FPS 43.578995490721695
Avg FPS 77.48630809247794
Avg FPS 135.1561850095736
Avg FPS 132.69971510332255
Avg FPS 139.59917254506735
Avg FPS 158.15940539030052
Avg FPS 171.37814381781214
Avg FPS 215.60676489643336
Avg FPS 250.05799970574333
Avg FPS 244.18068972467964
Avg FPS 240.376095162593
Avg FPS 230.78052654805484
Avg FPS 238.99090219777116
Avg FPS 226.70374036979072
Avg FPS 228.29986645501413
Avg FPS 235.100406267578
Avg FPS 250.92526924731325
Avg FPS 248.12398358094788
Avg FPS 248.34638855658451
Avg FPS 261.37994291516304
Avg FPS 272.7871857988728
Avg FPS 283.53616333519057
Avg FPS 293.93262162713205
Avg FPS 303.4925998437954
Avg FPS 311.40454638189493
Avg FPS 320.2187082218568
Avg FPS 333.0322936437307
Avg FPS 330.05793133012213
Avg FPS 330.1964299970918
Avg FPS 336.2253561130654
Avg FPS 336.30742219758133
Avg FPS 341.13146055105244
Avg FPS 340.79730122341033
Avg FPS 341.09874927521986
Avg FPS 331.6128117949215
Avg FPS 327.02341186311844
Avg FPS 324.1786793131247
Avg FPS 322.249

Avg FPS 406.4529892692592
Avg FPS 406.236962833704
Avg FPS 406.0654727511378
Avg FPS 406.42160529758263
Avg FPS 406.771553073193
Avg FPS 407.1237199819961
Avg FPS 407.57942683709115
Avg FPS 407.9134178254775
Avg FPS 408.2328538211254
Avg FPS 408.57747956798136
Avg FPS 409.61837121519545
Avg FPS 409.9985750774173
Avg FPS 410.34796940504106
Avg FPS 411.10142670755744
Avg FPS 410.8817602465473
Avg FPS 410.65335901279275
Avg FPS 410.4244169774477
Avg FPS 409.7437934959745
Avg FPS 409.52023452729617
Avg FPS 409.0505189949253
Avg FPS 408.4371977580737
Avg FPS 408.2198278066333
Avg FPS 408.00318392092936
Avg FPS 407.3977778145116
Avg FPS 406.537309213885
Avg FPS 406.3273489079438
Avg FPS 406.6016379502693
Avg FPS 406.8753825294793
Avg FPS 406.66649022934484
Avg FPS 406.45864246048103
Avg FPS 405.77480937093793
Avg FPS 405.33246205101403
Avg FPS 405.1296465374426
Avg FPS 405.40222879473384
Avg FPS 405.6724251150809
Avg FPS 405.47066556804987
Avg FPS 405.73915169341376
Avg FPS 405.5601458534593

Avg FPS 412.70321222303465
Avg FPS 412.82316681488845
Avg FPS 412.96211836314876
Avg FPS 412.8614984273827
Avg FPS 413.0471019174968
Avg FPS 413.2496915688237
Avg FPS 413.4528558959805
Avg FPS 413.58651695924453
Avg FPS 413.7911609023345
Avg FPS 413.1773473137291
Avg FPS 412.8520016835565
Avg FPS 412.52536856303493
Avg FPS 412.2769355905388
Avg FPS 412.1725516854344
Avg FPS 412.0524641500872
Avg FPS 411.81954800928986
Avg FPS 411.96841363536794
Avg FPS 412.10293609338447
Avg FPS 412.23859579119147
Avg FPS 412.38746402507553
Avg FPS 412.5226439984447
Avg FPS 412.669316557675
Avg FPS 412.79231389580326
Avg FPS 412.9268135123689
Avg FPS 413.0680377063265
Avg FPS 412.95552480435464
Avg FPS 412.8369065605469
Avg FPS 412.7240152051807
Avg FPS 412.85456076374044
Avg FPS 413.0499491640608
Avg FPS 413.181580912707
Avg FPS 413.2200626007158
Avg FPS 413.46952132411593
Avg FPS 413.3510176976931
Avg FPS 413.4818648003006
Avg FPS 413.9632183317072
Avg FPS 414.07951371818376
Avg FPS 414.2088798658416

Avg FPS 418.53061014636967
Avg FPS 418.7611934577822
Avg FPS 418.86894349560424
Avg FPS 418.97741701293444
Avg FPS 419.44405782754154
Avg FPS 419.5290960771364
Avg FPS 419.70632916987944
Avg FPS 419.5423282101089
Avg FPS 419.63923142899733
Avg FPS 419.7188781762201
Avg FPS 419.61716965291896
Avg FPS 419.70952687509094
Avg FPS 419.79251182566617
Avg FPS 419.8761861084902
Avg FPS 419.96590799556554
Avg FPS 420.0488651015876
Avg FPS 420.0001249774381
Avg FPS 420.08275513381125
Avg FPS 420.1632087164728
Avg FPS 420.0753629858154
Avg FPS 420.15748095082023
Avg FPS 420.2484734831679
Avg FPS 420.33103975404214
Avg FPS 420.4194787644618
Avg FPS 420.5017037786629
Avg FPS 420.59191529728923
Avg FPS 420.5138505170791
Avg FPS 420.5948823871096
Avg FPS 420.67635517147545
Avg FPS 420.77492773232393
Avg FPS 420.686049337525
Avg FPS 420.5996806453594
Avg FPS 420.5128135888217
Avg FPS 420.44280980204366
Avg FPS 420.25094325849506
Avg FPS 420.1837470921921
Avg FPS 420.11656635024246
Avg FPS 420.21471723

Avg FPS 425.9124918270684
Avg FPS 425.971197100432
Avg FPS 426.0293915337407
Avg FPS 426.01589166622443
Avg FPS 426.2600450763866
Avg FPS 426.318065277161
Avg FPS 426.24667496789215
Avg FPS 426.31208990170296
Avg FPS 426.3700740095376
Avg FPS 426.2985823896017
Avg FPS 426.2272632378825
Avg FPS 426.1559926496647
Avg FPS 426.22012329959057
Avg FPS 426.2780007843978
Avg FPS 426.2067739250596
Avg FPS 426.27100320323325
Avg FPS 426.3357065606041
Avg FPS 426.40026251682133
Avg FPS 426.4642899810224
Avg FPS 426.32947171423393
Avg FPS 426.19664288404124
Avg FPS 426.1296090144026
Avg FPS 425.9965170002398
Avg FPS 425.9264665868333
Avg FPS 425.85650301367457
Avg FPS 425.7937387360036
Avg FPS 425.7361664423021
Avg FPS 425.79498778906975
Avg FPS 425.7382872369398
Avg FPS 425.8275343106173
Avg FPS 425.9166988731883
Avg FPS 425.91459488057956
Avg FPS 426.00662743514346
Avg FPS 426.15048626712667
Avg FPS 425.9804174403816
Avg FPS 425.8090369854579
Avg FPS 425.6381564009395
Avg FPS 425.6190883130083
A

Avg FPS 428.64989667999066
Avg FPS 428.6937423488723
Avg FPS 428.7629716764803
Avg FPS 428.8330700677076
Avg FPS 428.8782808615315
Avg FPS 429.01503567936686
Avg FPS 429.10390716340237
Avg FPS 429.1483438134678
Avg FPS 429.09931719343183
Avg FPS 429.14105154053647
Avg FPS 429.0922904399847
Avg FPS 429.043881387704
Avg FPS 429.11207691455775
Avg FPS 429.18143774735677
Avg FPS 429.39552263353187
Avg FPS 429.46535818481294
Avg FPS 429.5343284649746
Avg FPS 429.77990199373943
Avg FPS 429.83368109018295
Avg FPS 429.99661107771703
Avg FPS 430.0405092122421
Avg FPS 430.08729877030044
Avg FPS 430.0304439977794
Avg FPS 430.0792255265059
Avg FPS 430.0201256361966
Avg FPS 429.96100026735826
Avg FPS 429.70243246796633
Avg FPS 429.6438040794285
Avg FPS 429.68729921438256
Avg FPS 429.7534553620485
Avg FPS 429.704683322756
Avg FPS 429.7486112810874
Avg FPS 429.70078377031535
Avg FPS 429.6786600051388
Avg FPS 429.840681138583
Avg FPS 429.90986235752575
Avg FPS 429.8623058736077
Avg FPS 429.60591066293

Avg FPS 435.5602715705457
Avg FPS 435.61467092340985
Avg FPS 435.66460213411005
Avg FPS 435.6699750097037
Avg FPS 435.7036870551867
Avg FPS 435.76865444424544
Avg FPS 435.7159909956392
Avg FPS 435.66305165399905
Avg FPS 435.69697626014664
Avg FPS 435.7308037704662
Avg FPS 435.7683024126682
Avg FPS 435.8066568452759
Avg FPS 435.84052998559423
Avg FPS 435.8783279947588
Avg FPS 435.9159594721788
Avg FPS 435.94972377759115
Avg FPS 435.98286736826503
Avg FPS 436.02524690048324
Avg FPS 436.18428967525233
Avg FPS 436.21812428883345
Avg FPS 436.2605811716688
Avg FPS 436.31423969689985
Avg FPS 436.3682376285879
Avg FPS 436.42200066177946
Avg FPS 436.4761032562018
Avg FPS 436.49657486562364
Avg FPS 436.5409149232675
Avg FPS 436.6126669639082
Avg FPS 436.63570014426347
Avg FPS 436.6884478005302
Avg FPS 436.56763058405517
Avg FPS 436.4468731550512
Avg FPS 436.3262196355551
Avg FPS 436.23132154723237
Avg FPS 436.1792296094864
Avg FPS 436.1270685044351
Avg FPS 436.1587822097181
Avg FPS 436.203412146

Avg FPS 440.85807678250103
Avg FPS 440.8938008356648
Avg FPS 440.94022253170147
Avg FPS 440.9671769938563
Avg FPS 440.9640124308732
Avg FPS 440.9162202152626
Avg FPS 440.9425097987228
Avg FPS 440.9693083793732
Avg FPS 440.99624323203506
Avg FPS 441.02307401792393
Avg FPS 441.05307286513255
Avg FPS 441.0833745922303
Avg FPS 441.1137318691423
Avg FPS 441.140515973852
Avg FPS 441.1710134531675
Avg FPS 441.3097303149392
Avg FPS 441.3366047952888
Avg FPS 441.14778198361086
Avg FPS 441.1192394294796
Avg FPS 441.0785567328075
Avg FPS 441.0306789623933
Avg FPS 440.9900586606839
Avg FPS 441.02059706476507
Avg FPS 441.053802420674
Avg FPS 441.02418745737793
Avg FPS 440.97719208876833
Avg FPS 440.92976728386424
Avg FPS 440.8842255651092
Avg FPS 440.8435389220699
Avg FPS 440.8123473056815
Avg FPS 440.76513756916887
Avg FPS 440.726947750857
Avg FPS 440.7696882284715
Avg FPS 440.8135509556311
Avg FPS 440.837817165867
Avg FPS 440.7908172479887
Avg FPS 440.8211460686009
Avg FPS 440.84767068054094
Avg 

#### pywin32 results:
When capturing the fullscreen window, the framerate sees an increase of almost 40% (actually closer to 36.8%, still pretty dam good) compared to using pillow! great news!

AAAAND, when capturing the small Samurai Gunn window, it goes to an avg of 44.24 fps! that is almost 200% increase!!!
and uhh... after cropping the black borders it went up to an avg of 790 FPS.... yeah, we are good with this.